<details><summary> </summary>

# Skip notebook test

</details>

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance
import notebook_utils.notebook_helpers as utils
from cuopt_thin_client import CuOptServiceClient

# Benchmark Gehring & Homberger
## Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)

While other notebooks such as [cvrptw_service_team_routing.ipynb](cvrptw_service_team_routing.ipynb) focus on the cuOpt API and high level problem modeling, here we focus on performance.

cuOpt offers a unique benefit over other solver_settingss, specifically, time to solution.  In addition to achieving world class accuracy, cuOpt also produces these solutions in a time frame that allows for re-optimization in dynamic environments and rapid iteration over possible problem configurations.

Here we are demonstrating this performance on a large popular academic [dataset by Gehing & Homberger](https://www.sintef.no/projectweb/top/vrptw/homberger-benchmark/).  These problems are well studied and used as the basis for comparison for VRP research and product offerings. The particular instance we will test with is from the group of largest (1000 location) problems.  Each problem instance has an associated best known solution, the one we will measure against is shown below

**API Reference**: [cuOpt Server Documentation](https://docs.nvidia.com/cuopt/serv_api.html)

In [ ]:
homberger_1000_file = 'notebook_utils/data/C1_10_1.TXT'

best_known_solution = {
    "n_vehicles": 100,
    "cost": 42478.95
}

### Problem Data
The data for this problem instance are provided via text file.  cuOpt has a utility function available specifically for the Gehring & Homberger benchmark which converts the problem into the components required by cuOpt.

In [ ]:
orders, vehicle_capacity, n_vehicles = utils.create_from_file(homberger_1000_file)
n_locations = orders["demand"].shape[0]-1
print("Number of locations          : ", n_locations)
print("Number of vehicles available : ", n_vehicles)
print("Capacity of each vehicle     : ", vehicle_capacity)
print("\nInitial Orders information")
print(orders)

# Initialize cuOpt Service Client and cuOpt Problem Data

In [ ]:
cuopt_client_id = os.environ["CUOPT_CLIENT_ID"]
cuopt_client_secret = os.environ["CUOPT_CLIENT_SECRET"]


cuopt_service_client = CuOptServiceClient(
    client_id=cuopt_client_id,
    client_secret=cuopt_client_secret,
    )

cuopt_problem_data = {}

### Cost Matrix

In [ ]:
coords = list(zip(orders['xcord'].to_list(),
                  orders['ycord'].to_list()))

cost_matrix = pd.DataFrame(distance.cdist(coords, coords, 'euclidean')).astype(np.float32).values.tolist()

### Set Cost Matrix

In [ ]:
cuopt_problem_data["cost_matrix_data"] = {
        "cost_matrix": {
            "0": cost_matrix
        }
    }

### Set Fleet Data

In [ ]:
vehicle_locations = [[0, 0]] * n_vehicles

cuopt_problem_data["fleet_data"] = {
        "vehicle_locations": vehicle_locations,
        "capacities": [[vehicle_capacity] * n_vehicles]
}

### Set Task Data

In [ ]:
cuopt_problem_data["task_data"] = {
        "task_locations": orders['vertex'].values.tolist(),
        "demand": [orders['demand'].values.tolist()],
        "task_time_windows": [list(i) for i in list(zip(orders['earliest_time'].values.tolist(),
                                                        orders['latest_time'].values.tolist()))],
        "service_times": orders['service_time'].values.tolist()
}

### Helper functions to solve and process the output

In [ ]:
def solution_eval(vehicles, cost, best_known_solution):
    
    print(f"- cuOpt provides a solution using {vehicles} vehicles")
    print(f"- This represents {vehicles - best_known_solution['n_vehicles']} more than the best known solution")
    print(f"- Vehicle Percent Difference {(vehicles/best_known_solution['n_vehicles'] - 1)*100}% \n\n")
    print(f"- In addition cuOpt provides a solution cost of {cost}") 
    print(f"- Best known solution cost is {best_known_solution['cost']}")
    print(f"- Cost Percent Difference {(cost/best_known_solution['cost'] - 1)*100}%")

### Get Optimized Results

Update solver config and test different run-time 

**1 Second Time Limit**

Note : due to the large amount of data network transfer time can exceed the requested solve time.

In [ ]:
cuopt_problem_data["solver_config"] = {
        "time_limit": 5
    }

# Solve the problem
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

In [ ]:
# Evaluation:
solution_eval(solver_resp["num_vehicles"], solver_resp["solution_cost"], best_known_solution)

**10 Second Time Limit**

In [ ]:
cuopt_problem_data["solver_config"] = {
        "time_limit": 10.0
    }

# Solve the problem
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

In [ ]:
# Evaluation:
solution_eval(solver_resp["num_vehicles"], solver_resp["solution_cost"], best_known_solution)

**20 Second Time Limit**

In [ ]:
cuopt_problem_data["solver_config"] = {
        "time_limit": 20.0
    }

# Solve the problem
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

In [ ]:
# Evaluation:
solution_eval(solver_resp["num_vehicles"], solver_resp["solution_cost"], best_known_solution)